In [48]:
import csv
import torch
from datasets import Dataset
import transformers
from transformers import (
  AdamW,
  BertConfig,
  BertModel,
  BertTokenizer,
  DistilBertTokenizer,
  DistilBertModel,
  DistilBertForSequenceClassification,
  BertForSequenceClassification)
from torch.utils.data import DataLoader
import torch.nn as nn
import os

In [7]:
finegrained_sentiments_dict = {
"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"]
}

In [8]:
!ls ../

input  lib  working


In [9]:
DATA_DIR = "../input/emotionclss/"
train = {"input": [], "labels": []}
dev = {"input": [], "labels": []}
test = {"input": [], "labels": []}

with open(DATA_DIR + "train.tsv") as file:
    tsv_file = csv.reader(file, delimiter="\t") 
    for line in tsv_file:
        train["input"].append(line[0])
        labels = line[1].split(",")
        one_hot = [0 for i in range(28)]
        for label in labels:
            one_hot[int(label)] = 1
        train["labels"].append(one_hot)

with open(DATA_DIR + "dev.tsv") as file:
    tsv_file = csv.reader(file, delimiter="\t") 
    for line in tsv_file:
        dev["input"].append(line[0])
        labels = line[1].split(",")
        one_hot = [0 for i in range(28)]
        for label in labels:
            one_hot[int(label)] = 1
        dev["labels"].append(one_hot)

with open(DATA_DIR + "test.tsv") as file:
    tsv_file = csv.reader(file, delimiter="\t") 
    for line in tsv_file:
        test["input"].append(line[0])
        labels = line[1].split(",")
        one_hot = [0 for i in range(28)]
        for label in labels:
            one_hot[int(label)] = 1
        test["labels"].append(one_hot)
        
print("Number of train examples are {}".format(len(train["input"])))
print("Number of dev examples are {}".format(len(dev["input"])))
print("Number of test examples are {}".format(len(test["input"])))

Number of train examples are 43410
Number of dev examples are 5426
Number of test examples are 5427


In [10]:
# Creating higgingface datasets
train_dataset = Dataset.from_dict(train)
dev_dataset = Dataset.from_dict(dev)
test_dataset = Dataset.from_dict(test)

print(train_dataset)

Dataset({
    features: ['input', 'labels'],
    num_rows: 43410
})


In [11]:
from torch.utils.data import Dataset
class LoadData(Dataset):
    """
    Using this since dataloader expects map-style dataset objects
    
    """
    
    def __init__(
        self, dataset, tokenizer, source_length):
        """
        Initializes a Dataset class

        Args:
            dataset (Dataset object): Input Dataset
            tokenizer (Tokenizer object): Transformer tokenizer
            source_length (int): Max length of source text
        """
        
        self.tokenizer = tokenizer
        self.data = dataset
        self.source_length = source_length
        self.source_text = self.data["input"]
        self.target_labels = self.data["labels"]

    def __len__(self):
        return len(self.target_labels)

    def __getitem__(self, index):
        """
        return input ids, attention masks and target ids
        
        """
        source_text = str(self.source_text[index])
        target_label = self.target_labels[index]

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())

        source = self.tokenizer.__call__(
            [source_text],
            max_length=self.source_length,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        
        target = torch.tensor(target_label)

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target": target.squeeze().to(dtype=torch.long)
        }

In [45]:
#joeddav/distilbert-base-uncased-go-emotions-student
parameters = {"model": "bhadresh-savani/bert-base-go-emotion",  # model_type: t5-base/t5-large
    "train_bs": 8,  # training batch size
    "val_bs": 10,  # validation batch size
    "test_bs": 15,
    "epochs": 3,  # number of training epochs
    "lr": 6e-4,  # learning rate
    "wd": 0.0001,
    "max_source_length": 512,  # max length of source text
    "SEED": 42,
    "out_dir": "./",
    "hidden_size": 768,
    "num_classes": 28}

index_label = {0:"admiration", 1:"amusement", 2:"anger", 3:"annoyance", 4:"approval", 5:"caring", 6:"confusion",
            7:"curiosity", 8:"desire", 9:"disappointment", 10:"disapproval", 11:"disgust", 12:"embarrassment",
            13:"excitement", 14:"fear", 15:"gratitude", 16:"grief", 17:"joy", 18:"love", 19:"nervousness",
            20:"optimism", 21:"pride", 22:"realization", 23:"relief", 24:"remorse", 25:"sadness",
            26:"surprise", 27:"neutral"}
label_list = list(index_label.values())

In [79]:
# compute metrics on test data
def compute_metrics_allemotions(outputs, labels, label_list, index_label):
    predictions = []
    
    for output in outputs:
        output = [int(out > 0.32) for out in output]
        predictions.append(output)
    print("1st prediction", predictions[0])
    
    confusion_matrix = {}
    precisions, recalls, fscores = {}, {}, {}
    for label in label_list:
        confusion_matrix[label] = {"TP":0, "FP": 0, "FN": 0}
        precisions[label], recalls[label], fscores[label] = 0, 0, 0
    
    for i, prediction in enumerate(predictions):
        gt = labels[i]
        for j, out in enumerate(gt):
            pred = prediction[j]
            if out == 0 and pred == 0: continue
            elif out == 0 and pred == 1:
                # FP found
                confusion_matrix[index_label[j]]["FP"] += 1
            elif out == 1 and pred == 0:
                # FN found
                confusion_matrix[index_label[j]]["FN"] += 1
            elif out == 1 and pred == 1:
                # TP found
                confusion_matrix[index_label[j]]["TP"] += 1
    
    
    for label in label_list:
        precisions[label] = confusion_matrix[label]["TP"]/(confusion_matrix[label]["TP"] + confusion_matrix[label]["FP"] + 1e-4)
        recalls[label] = confusion_matrix[label]["TP"]/(confusion_matrix[label]["TP"] + confusion_matrix[label]["FN"] + 1e-4)
        fscores[label] = 2*precisions[label]*recalls[label]/(precisions[label]+recalls[label] + 1e-4)
    
    return precisions, recalls, fscores

    

def compute_test_outputs(model, test_dataloader, tokenizer, device, label_list, index_label):
    predictions = []
    labels = []
    
    with torch.no_grad():
        steps = 0
        for test_batch in test_dataloader:
            y = test_batch['target'].to(device, dtype = torch.float32)
            ids = test_batch['source_ids'].to(device, dtype = torch.long)
            mask = test_batch['source_mask'].to(device, dtype = torch.long)

            output = model(
                input_ids=ids,
                attention_mask=mask,
            )
            
            output = output["logits"]
            output = torch.sigmoid(output)
            
            predictions.extend(output.detach().cpu().numpy())
            labels.extend(y.detach().cpu().numpy())
            if steps == 5: break
    
    return predictions, labels

In [49]:
cuda =  torch.cuda.is_available()
device = torch.device("cuda") if cuda else torch.device("cpu")

tokenizer = DistilBertTokenizer.from_pretrained(parameters["model"])
model = BertForSequenceClassification.from_pretrained(parameters["model"])

In [50]:
model = model.to(device)
test_obj = LoadData(
        test_dataset,
        tokenizer,
        parameters["max_source_length"]
    )
test_loader = DataLoader(test_obj, shuffle=True, batch_size=parameters["test_bs"])
predictions, labels = compute_test_outputs(model, test_loader, tokenizer, device, label_list, index_label)

In [77]:
dev_obj = LoadData(
        dev_dataset,
        tokenizer,
        parameters["max_source_length"]
    )
dev_loader = DataLoader(dev_obj, shuffle=True, batch_size=parameters["val_bs"])
dev_predictions, dev_labels = compute_test_outputs(model, dev_loader, tokenizer, device, label_list, index_label)

In [80]:
precisions, recalls, fscores = compute_metrics_allemotions(predictions, labels, label_list, index_label)
print("Precision, Recall and Fscores for all labels are ")

precision, recall, fscore = 0, 0, 0
for label in label_list:
    precision += precisions[label]
    recall += recalls[label]
    fscore += fscores[label]
    print("Emotion {}: precision: {}, recall: {}, fscore: {}".format(label, precisions[label], 
                                                                     recalls[label], fscores[label]))

print("Macro precision: {}, Macro recall: {}, Macro fscore: {}".format(precision/28, recall/28, fscore/28))

1st prediction [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Precision, Recall and Fscores for all labels are 
Emotion admiration: precision: 0.7300612003964826, recall: 0.7083331927910332, fscore: 0.7189831026922668
Emotion amusement: precision: 0.7641507030972632, recall: 0.9204541967976528, fscore: 0.8350016928135315
Emotion anger: precision: 0.5902773678629389, recall: 0.42929271247842804, fscore: 0.49702698402148593
Emotion annoyance: precision: 0.5662643779947253, recall: 0.14687495410157686, fscore: 0.23321780161193786
Emotion approval: precision: 0.583332928241022, recall: 0.23931617113499398, fscore: 0.3393525510741509
Emotion caring: precision: 0.5396816830449476, recall: 0.25185166529506275, fscore: 0.343390613569084
Emotion confusion: precision: 0.6206885850196809, recall: 0.23529396386015433, fscore: 0.34119204434929035
Emotion curiosity: precision: 0.5342463923813725, recall: 0.5492955812339503, fscore: 0.5416164928462633
Emotion des

In [81]:
precisions, recalls, fscores = compute_metrics_allemotions(dev_predictions, dev_labels, label_list, index_label)
print("Precision, Recall and Fscores for all labels are ")

precision, recall, fscore = 0, 0, 0
for label in label_list:
    precision += precisions[label]
    recall += recalls[label]
    fscore += fscores[label]
    print("Emotion {}: precision: {}, recall: {}, fscore: {}".format(label, precisions[label], 
                                                                     recalls[label], fscores[label]))

print("Macro precision: {}, Macro recall: {}, Macro fscore: {}".format(precision/28, recall/28, fscore/28))

1st prediction [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Precision, Recall and Fscores for all labels are 
Emotion admiration: precision: 0.7499998437500326, recall: 0.7377047668637773, fscore: 0.7437515059900589
Emotion amusement: precision: 0.7387638374258885, recall: 0.8679865122156726, fscore: 0.7981291434406489
Emotion anger: precision: 0.5548383517171924, recall: 0.4410254148587616, fscore: 0.4913789486275758
Emotion annoyance: precision: 0.48051885646901754, recall: 0.12211217092007562, fscore: 0.19470443055052772
Emotion approval: precision: 0.700786849774134, recall: 0.2241813037326691, fscore: 0.33965780580710503
Emotion caring: precision: 0.6756747626016721, recall: 0.32679717202799213, fscore: 0.440484306416327
Emotion confusion: precision: 0.6727260495890007, recall: 0.24342089248625493, fscore: 0.3574485607969584
Emotion curiosity: precision: 0.5139440183489966, recall: 0.5201610805802095, fscore: 0.5169838675499486
Emotion desir